https://x.com/littmath/status/1786115416231641335  

<img src="img/HTTTH_vs_HTHTH.png"></img>

In [6]:
import random
import numpy as np

In [47]:
def flip_coin():
    return 'H' if random.random() < 0.5 else 'T'

def monte_carlo_simulation(trials):
    pattern1 = "HTTTH"
    pattern2 = "HTHTH"
    pattern1_flips = []
    pattern2_flips = []

    for _ in range(trials):
        flips = ""
        flip_count = 0
        while True:
            flips += flip_coin()
            flip_count += 1
            if len(flips) >= 5:
                if flips[-5:] == pattern1:
                    pattern1_flips.append(flip_count)
                    break
                elif flips[-5:] == pattern2:
                    pattern2_flips.append(flip_count)
                    break

    return pattern1_flips, pattern2_flips


In [50]:
trials = 100_000
pattern1_flips, pattern2_flips = monte_carlo_simulation(trials)

mean_flips_pattern1 = sum(pattern1_flips) / len(pattern1_flips) if pattern1_flips else 0
mean_flips_pattern2 = sum(pattern2_flips) / len(pattern2_flips) if pattern2_flips else 0

pattern1_count = len(pattern1_flips)
pattern2_count = len(pattern2_flips)

print(f"Pattern 'HTTTH' occurred {pattern1_count} times with a mean of {mean_flips_pattern1:.2f} flips.")
print(f"Pattern 'HTHTH' occurred {pattern2_count} times with a mean of {mean_flips_pattern2:.2f} flips.")

Pattern 'HTTTH' occurred 55620 times with a mean of 19.90 flips.
Pattern 'HTHTH' occurred 44380 times with a mean of 19.45 flips.


# Markov Chain

In [45]:
states = ["", "H", "T", "HT", "HTH", "HTT", "HTHT", "HTTT", "HTHTH", "HTTTH"]

# Create the transition matrix
transition_matrix = np.zeros((len(states), len(states)))

# Fill the transition matrix
for i, state in enumerate(states):
    if state == "HTTTH" or state == "HTHTH":
        transition_matrix[i][i] = 1.0
    else:
        next_state_H = (state + "H")[-5:]
        next_state_T = (state + "T")[-5:]
        if next_state_H in states:
            transition_matrix[i][states.index(next_state_H)] = 0.5
        else:
            # go back to starting 'H' state
            transition_matrix[i][1] = 0.5
        if next_state_T in states:
            transition_matrix[i][states.index(next_state_T)] = 0.5
        else:
            # go back to starting 'T' state
            transition_matrix[i][2] = 0.5

# Manual update of getting 'T' with HTHT, and going back to HTT
transition_matrix[6][2] = 0
transition_matrix[6][5] = .5

# Raise the matrix to a large power to find the steady-state probabilities
n = 1000 
steady_state_matrix = np.linalg.matrix_power(transition_matrix, n)

# The steady-state probabilities are in the first row
steady_state_probabilities = steady_state_matrix[0]

# Extract the probabilities for the patterns
pattern1_prob = steady_state_probabilities[states.index("HTTTH")]
pattern2_prob = steady_state_probabilities[states.index("HTHTH")]

# Print the transition matrix
print("Transition Matrix:")
print(transition_matrix)
print()

print(f"Steady-state probability for 'HTTTH': {pattern1_prob}")
print(f"Steady-state probability for 'HTHTH': {pattern2_prob}")
print(f"Ratio (HTTTH/HTHTH): {pattern1_prob / pattern2_prob if pattern2_prob != 0 else 'undefined'}")

Transition Matrix:
[[0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.  0.5 0. ]
 [0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]

Steady-state probability for 'HTTTH': 0.5555555555555556
Steady-state probability for 'HTHTH': 0.4444444444444445
Ratio (HTTTH/HTHTH): 1.25
